In [1]:
import findspark
# my local spark install
findspark.init('/Users/dreyco676/spark-1.6.0-bin-hadoop2.6/')

import pyspark
from pyspark.sql import SQLContext
from pyspark.sql.types import *

sc = pyspark.SparkContext()
sqlContext = SQLContext(sc)

In [2]:
# UNCOMMENT TO SKIP PREPROCESSING AND START WITH CLEAN DATA!!!
def reload_checkpoint(data_rdd):
    parts_rdd = data_rdd.map(lambda l: l.split("\t"))
    # Filter bad rows out
    garantee_col_rdd = parts_rdd.filter(lambda l: len(l) == 3)
    typed_rdd = garantee_col_rdd.map(lambda p: (p[0], p[1], float(p[2])))
    # Create DataFrame
    df = sqlContext.createDataFrame(typed_rdd, ["id", "text", "label"])
    return df


# Load precleaned training set
training_rdd = sc.textFile("data/clean_training.txt")
training_df = reload_checkpoint(training_rdd)
# Load precleaned test set
test_rdd = sc.textFile("data/clean_test.txt")
test_df = reload_checkpoint(test_rdd)

In [3]:
clean_df = training_df.unionAll(test_df)

In [4]:
# UNCOMMENT TO SKIP PREPROCESSING AND START WITH CLEAN DATA!!!
raw_rdd = sc.textFile("data/raw_classified.txt")
parts_rdd = raw_rdd.map(lambda l: l.split("\t"))
# Filter bad rows out
garantee_col_rdd = parts_rdd.filter(lambda l: len(l) == 3)
typed_rdd = garantee_col_rdd.map(lambda p: (p[0], p[1], p[2]))
# Create DataFrame
raw_df = sqlContext.createDataFrame(typed_rdd, ["raw_text", "raw_id", "raw_label"])

In [5]:
new_raw_df = raw_df.join(clean_df, raw_df.raw_id == clean_df.id)

In [6]:
final_raw_df = new_raw_df.select("id", "raw_text", "label")

In [8]:
save_format = final_raw_df.map(lambda row: row[1]+'\t'+row[0]+'\t'+str(row[2]))
save_format.saveAsTextFile('data/raw_classified')